In [189]:
import pandas as pd
import gspread
import os
from datetime import datetime

In [28]:
# Inicia serviço do Google Sheets
gs = gspread.service_account(filename='service_account.json')

In [423]:
orcamentos = gs.open_by_key('14d0b2l65tMKZll6SAlqXqRqAjIEIHwkRhJI4YVgIi88')
materiais_orcados = orcamentos.worksheet('MATERIAIS').get_all_records(value_render_option="UNFORMATTED_VALUE")
materiais_orcados = pd.DataFrame(materiais_orcados)
materiais_orcados = materiais_orcados.loc[materiais_orcados['Projeto'] != '']   
materiais_orcados = materiais_orcados.rename(columns = {'Código': 'Material'})

In [33]:
carteira = gs.open_by_key('1yZUYH8x-LzlA7OqrExA7igqin1PG8h3TWQPt0UUJqxI')
carteira = carteira.worksheet('CARTEIRA').get_all_records()
carteira = pd.DataFrame(carteira)
carteira = carteira.loc[carteira['PROJETO'] != '']
carteira = carteira.rename(columns={'PROJETO':'Projeto'})
projeto_vs_unidade = carteira.groupby(['Projeto', 'UNIDADE'], as_index=False).count()[['Projeto', 'UNIDADE']]

In [38]:
programacao = gs.open_by_key('1ztV6DYCUkhefULyxJTBiaAKLB_x5zAHgK9icFqBDvf4')
programacao = programacao.worksheet('PROGRAMAÇÃO').get_all_records()
programacao = pd.DataFrame(programacao)
programacao = programacao.loc[programacao['Projeto'] != '']
programacao['Data Execução'] = pd.to_datetime(programacao['Data Execução'], format='%d/%m/%Y')

In [497]:
try:
    os.rename('cji3.xls', 'cji3.csv')
except FileExistsError:
    os.remove('cji3.csv')
    os.rename('cji3.xls', 'cji3.csv')
except FileNotFoundError:
    pass

cji3 = pd.read_csv('cji3.csv', sep='\t', encoding='ISO-8859-1', skiprows=1)
cji3 = cji3.loc[cji3['Unnamed: 1'] != '*']
cji3['Material'] = cji3['Material'].fillna(0)
cji3['Material'] = cji3['Material'].astype(int)
cji3 = cji3.loc[cji3['Material'] != 0]
cji3 = cji3.loc[cji3['Usuário'].str.contains('STC|SIR')]

cji3['  Qtd.entr.'] = cji3['  Qtd.entr.'].str.replace('.','')
cji3['  Qtd.entr.'] = cji3['  Qtd.entr.'].str.replace(',','.')
cji3['  Qtd.entr.'] = cji3['  Qtd.entr.'].str.replace(' ','')
cji3['  Qtd.entr.'] = cji3['  Qtd.entr.'].astype(float)

cji3['Def.proj.'] = cji3['Def.proj.'].str.extract(r'(\d\d\d\d\d\d\d)').astype(int)

In [444]:
try:
    os.rename('zmm370.xls', 'zmm370.csv')
except FileExistsError:
    os.remove('zmm370.csv')
    os.rename('zmm370.xls', 'zmm370.csv')
except FileNotFoundError:
    pass

zmm370 = pd.read_csv('zmm370.csv', sep='\t', encoding='ISO-8859-1', skiprows=1)
zmm370 = zmm370.loc[zmm370['Registro Final'] != 'X']
zmm370 = zmm370.loc[(zmm370['Tipo movimento'] == 221) | (zmm370['Tipo movimento'] == 921)]
zmm370 = zmm370.loc[zmm370['Recebedor'].str.contains('3ST')]

zmm370['Qtd Necessária'] = zmm370['Qtd Necessária'].str.replace('.','')
zmm370['Qtd Necessária'] = zmm370['Qtd Necessária'].str.replace(',','.')
zmm370['Qtd Necessária'] = zmm370['Qtd Necessária'].str.replace(' ','')
zmm370['Qtd Necessária'] = zmm370['Qtd Necessária'].astype(float)

zmm370['Qtd Pendente'] = zmm370['Qtd Pendente'].str.replace('.','')
zmm370['Qtd Pendente'] = zmm370['Qtd Pendente'].str.replace(',','.')
zmm370['Qtd Pendente'] = zmm370['Qtd Pendente'].str.replace(' ','')
zmm370['Qtd Pendente'] = zmm370['Qtd Pendente'].astype(float)

zmm370['Projeto'] = zmm370['Projeto'].str.extract(r'(\d\d\d\d\d\d\d)').astype(int)
zmm370 = zmm370.rename(columns={'Qtd Pendente': 'Qtd. Reservada'})

C:\Users\Hugo Viana\AppData\Local\Temp\ipykernel_30532\1998791053.py:8: DtypeWarning: Columns (0,4,6,7,11,12,16,18,20,23) have mixed types. Specify dtype option on import or set low_memory=False.
  zmm370 = pd.read_csv('zmm370.csv', sep='\t', encoding='ISO-8859-1', skiprows=1)


In [523]:
reservas_criadas = zmm370[['Projeto', 'Material', 'Qtd. Reservada']].groupby(['Projeto', 'Material'], as_index=False).sum()
materiais_movimentados = cji3[['Def.proj.','Material', '  Qtd.entr.']].groupby(['Def.proj.', 'Material'], as_index=False).sum()
materiais_movimentados = materiais_movimentados.rename(columns={'Def.proj.': 'Projeto', '  Qtd.entr.': 'Qtd. Movimentada'})

In [524]:
data_inicio = datetime.strptime('2023-04-17', '%Y-%m-%d')
data_fim = datetime.strptime('2023-04-22', '%Y-%m-%d')

programacao_semana = programacao.loc[(programacao['Data Execução']>=data_inicio) & (programacao['Data Execução']<=data_fim)]
obras_programadas = programacao_semana[['Projeto']].groupby('Projeto').sum()

material_obras_programadas = obras_programadas.merge(materiais_orcados, on='Projeto', how='left').groupby(['Projeto', 'Material'], as_index=False).sum()
material_obras_programadas = material_obras_programadas.rename(columns={'Quantidade': 'Qtd. Orçada'})
material_obras_programadas = material_obras_programadas.merge(materiais_movimentados, on=['Projeto', 'Material'], how='left')
material_obras_programadas = material_obras_programadas.merge(reservas_criadas, on=['Projeto', 'Material'], how='left')
material_obras_programadas = material_obras_programadas.fillna(0)
material_obras_programadas['Material pendente de saque'] = material_obras_programadas['Qtd. Orçada'] - material_obras_programadas['Qtd. Movimentada']
material_obras_programadas = material_obras_programadas.loc[material_obras_programadas['Material pendente de saque'] > 0]
material_obras_programadas['Reserva pendente'] = material_obras_programadas['Material pendente de saque'] - material_obras_programadas['Qtd. Reservada']
material_obras_programadas = material_obras_programadas.loc[material_obras_programadas['Reserva pendente'] > 0]
material_obras_programadas = material_obras_programadas.merge(projeto_vs_unidade[['Projeto', 'UNIDADE']], on='Projeto')

In [533]:
material_obras_programadas.to_excel('texte.xlsx')